# Upgraded Nonlinear System Analysis
Interactive notebook for 2D/3D nonlinear systems with multiple Lyapunov functions, bifurcation sweeps, and sliders.

## 1. User Inputs
Set system type, parameters, initial conditions, and choose equations.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider, IntSlider

# --- System Type ---
system_type = '2D'  # '2D' or '3D'

# --- Parameters ---
alpha = 1.1
beta = 0.4
delta = 0.1
gamma = 0.4

# --- Initial Conditions ---
initial_conditions_2D = [(5,2),(10,5),(15,10)]
initial_conditions_3D = [(5,2,1),(10,5,3)]

# --- System Equations ---
def system_2D(t, z, alpha, beta, delta, gamma):
    x, y = z
    dxdt = alpha*x - beta*x*y
    dydt = delta*x*y - gamma*y
    return [dxdt, dydt]

def system_3D(t, z, alpha, beta, delta, gamma):
    x, y, z_var = z
    dxdt = alpha*x - beta*x*y
    dydt = delta*x*y - gamma*y
    dzdt = -0.1*z_var + 0.05*x*y  # example extra equation
    return [dxdt, dydt, dzdt]


## 2. Fixed Points & Stability
Compute equilibria and Jacobian eigenvalues for stability.

In [ ]:

# Example fixed points for 2D Lotka-Volterra
x_star = gamma/delta
y_star = alpha/beta
print(f'2D fixed point: ({x_star}, {y_star})')

# Jacobian
J = np.array([[alpha - beta*y_star, -beta*x_star],
              [delta*y_star, delta*x_star - gamma]])
eigvals = np.linalg.eigvals(J)
print('Eigenvalues:', eigvals)


## 3. Multiple Lyapunov Functions
Define multiple candidate functions and plot level sets.

In [ ]:

x_vals = np.linspace(0.1, 2*x_star, 200)
y_vals = np.linspace(0.1, 2*y_star, 200)
X, Y = np.meshgrid(x_vals, y_vals)

V1 = delta*(X - x_star - x_star*np.log(X/x_star)) + beta*(Y - y_star - y_star*np.log(Y/y_star))
V2 = 0.5*(X-x_star)**2 + 0.3*(Y-y_star)**2

plt.figure(figsize=(7,5))
plt.contourf(X, Y, V1, levels=50, cmap='viridis')
plt.contour(X, Y, V2, levels=20, colors='red')
plt.plot(x_star, y_star, 'ro', label='Fixed Point')
plt.xlabel('x'); plt.ylabel('y'); plt.title('Lyapunov Functions V1 (filled) & V2 (contours)')
plt.legend(); plt.show()


## 4. Phase Portrait & Vector Field
Interactive plot with sliders for initial conditions.

In [ ]:

def plot_phase(alpha_val, beta_val):
    x_vals = np.linspace(0, 2*x_star, 20)
    y_vals = np.linspace(0, 2*y_star, 20)
    X, Y = np.meshgrid(x_vals, y_vals)
    U = alpha_val*X - beta_val*X*Y
    V_field = delta*X*Y - gamma*Y
    plt.figure(figsize=(7,6))
    plt.quiver(X, Y, U, V_field, color='gray')
    t_span = (0,50)
    for ic in initial_conditions_2D:
        sol = solve_ivp(system_2D, t_span, ic, args=(alpha_val, beta_val, delta, gamma), t_eval=np.linspace(*t_span,500))
        plt.plot(sol.y[0], sol.y[1], label=f'IC {ic}')
    plt.xlabel('x'); plt.ylabel('y'); plt.title('Phase Portrait with Vector Field')
    plt.grid(True); plt.legend(); plt.show()

interact(plot_phase, alpha_val=FloatSlider(min=0.1,max=2,value=alpha,step=0.05),
                     beta_val=FloatSlider(min=0.1,max=1,value=beta,step=0.05))


## 5. Automatic Bifurcation Sweep
Sweep alpha to see how y* changes automatically.

In [ ]:

alphas = np.linspace(0.1,2,100)
y_fp = alphas / beta
plt.figure(figsize=(8,5))
plt.plot(alphas, y_fp, label='y* vs alpha')
plt.xlabel('alpha'); plt.ylabel('y*'); plt.title('Automatic Bifurcation Sweep')
plt.grid(True); plt.legend(); plt.show()


## 6. Optional Export
You can save trajectory data, Lyapunov surfaces, or plots using np.savetxt or plt.savefig().